In [1]:
from bs4 import BeautifulSoup as bs
import requests
from typing import List, Dict, Tuple
import json
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import re


In [33]:
states[5]

{'BAYELSA': 'https://www.inecelectionresults.ng//elections/63f8f25b594e164f8146a213?state=6'}

In [2]:
links = []

In [3]:
def get_states_links(driver: webdriver) -> List[Dict[str,str]]:
    """
    Retrieves links for each state for the 2023 Nigerian presidential election.
    
    Args:
    - driver: WebDriver object from Selenium.
    
    Returns:
    - A list of dictionaries containing state name and its corresponding link.
    
    """
    url = "https://www.inecelectionresults.ng/pres/elections/63f8f25b594e164f8146a213?type=pres"
    driver.get(url)
    time.sleep(15)
    soup = bs(driver.page_source,'lxml')
    states = {i.find('a').text:"https://www.inecelectionresults.ng/" +i.find('a')['href'] for i in soup.find('div',{'class':"col-md-9"}).find_all('div',{'class':'m-2 p-2 bg-light'})}
    return states

def get_lga_link():
    """
    Returns:
    - A list of dictionaries containing lgas name and its corresponding link for the current state.
    
    """
    time.sleep(20)
    soup = bs(driver.page_source,'lxml')
    lgas = {i.find('a').text:"https://www.inecelectionresults.ng/" +i.find('a')['href'] for i in soup.find_all('div',{'class':re.compile('m-2 p-2 bg-light')})}
    
    return lgas 

def get_lgas_links(driver: webdriver, state_url: str) -> List[Dict[str,str]]:
    """
    Retrieves all lgas links for the current state
    
    Args:
    - driver: WebDriver object from Selenium.
    - state_url: state link
    
    Returns:
    - A list of dictionaries containing lgas name and its corresponding link.
    
    """
    driver.get(state_url)
    lgas = get_lga_link()
    #in case the page didnt load before it tries to extract, it will retry the process (reason why we have two
    #functions for this)
    if len(lgas) <1:
        lgas = get_lga_link()
    return lgas


def get_wards_links(driver: webdriver, lga_url: str) -> List[Dict[str,str]]:
    """
    Retrieves all wards links for the current state and lga
    
    Args:
    - driver: WebDriver object from Selenium.
    - lga_url: lga link
    
    Returns:
    - A list of dictionaries containing wards name and its corresponding link.
    
    """
    driver.get(lga_url)
    time.sleep(15)
    soup = bs(driver.page_source,'lxml')
    wards = {i.find('a').text:"https://www.inecelectionresults.ng/" +i.find('a')['href'] for i in soup.find_all('div',{'class':re.compile('m-2 p-2 bg-light')})}
    return wards

def poll_unit_ward_section():
    time.sleep(10)
    #find the parent element that contains all PUs
    html_ward = driver.find_element(By.XPATH, '/html/body/app-root/div/app-activated/div/div/div/div/app-election-lga/div/div/div/div/div[2]/div/div[2]/div[2]/div').get_attribute('innerHTML')
    #find the container
    div = bs(html_ward,'lxml').find('body')
    #get immediate child of thr parent element
    pus = div.findAll("div" , recursive=False)
    len_pu = len(pus)
    return pus

def file_src():
    """
    Returns:
    - the link to the file
    
    """
    time.sleep(5)
    soup = bs(driver.page_source,'lxml')
    urls = soup.iframe['src']
    return urls

def download_file(i):

    filename = list(i.keys())[0].replace(':','').replace('/','')
    
    url_file = list(i.values())[0]
    extension = url_file.rsplit('.')[-1]
    response = requests.get(url_file)
    
    new_filename = f"{filename}.{extension}"
    with open(new_filename, "wb")as f:
        f.write(response.content)

def scrape_all_pus_links(driver: webdriver, ward_url: str):
    global links
    driver.get(ward_url)
    time.sleep(20)
    #gets all PUs
    try:
        pus = poll_unit_ward_section()
    except:
        pus = poll_unit_ward_section()
    #loop through each PU , selenium is used to click the button using the index
    for idx,q in enumerate(pus):
        try:
            pu = ' '.join(q.text.split()[:-2])
            current_url = driver.current_url
            print(pu)
            try:
                time.sleep(10)
                driver.find_element(By.XPATH,f'/html/body/app-root/div/app-activated/div/div/div/div/app-election-lga/div/div/div/div/div[2]/div/div[2]/div[2]/div/div[{idx+1}]/div/div[2]/button').click()
            except:
                time.sleep(15)
                driver.find_element(By.XPATH,f'/html/body/app-root/div/app-activated/div/div/div/div/app-election-lga/div/div/div/div/div[2]/div/div[2]/div[2]/div/div[{idx+1}]/div/div[2]/button').click()
            time.sleep(10)
            try:
                urls = file_src()
            except:

                urls = file_src()
            
#             download_file({pu:urls})
            links.append({pu:urls})
            
        except:
            print(f"{q.text} count not be gotten")
        driver.get(current_url)


In [7]:
def get_lga_data(i):
    lgas = get_lgas_links(driver,i)
    for k in lgas.keys():
        wards = get_wards_links(driver,lgas[k])
        for j in wards.keys():
            x = scrape_all_pus_links(driver,wards[j])
        break
    

In [8]:
def main(single=None):
    global driver
    #open the webdriver
    driver= webdriver.Chrome()
    states = get_states_links(driver)
    if single:
        for i in states.keys():
            get_lga_data(states[i])
    else:
        if single.upper in states:
            get_lga_data(states[single.upper()])
    

In [ ]:
main('BAYELSA')

ABIA PLOY XVIII PU Code: 01/01/01/035
EZIAMA HIGH SCH. XI PU Code: 01/01/01/050
ABIA PLOY XXVII PU Code: 01/01/01/044
 ABIA PLOY XXVII PU Code: 01/01/01/044 View result count not be gotten
ABIA POLY XXVI PU Code: 01/01/01/043
ABIA PLOY XXV PU Code: 01/01/01/042
ABIA PLOY XXIV PU Code: 01/01/01/041
 ABIA PLOY XXIV PU Code: 01/01/01/041 View result count not be gotten
ABIA PLOY XX PU Code: 01/01/01/037
 ABIA PLOY XX PU Code: 01/01/01/037 View result count not be gotten
ABIA POLY XV PU Code: 01/01/01/032


In [ ]:
main()

In [8]:
def download_all():
    for link in links:
        download_file(link)
        print('downloaded ',link.keys() )

In [9]:
download_all()

downloaded  dict_keys(['RESTORATION DRIVE SQUARE PU Code: 06/04/01/018'])
downloaded  dict_keys(['COMMUNITY GIRLS SEC SCHOOL, NEMBE PU Code: 06/04/01/017'])
downloaded  dict_keys(['AMABEBE-OPO SQUARE PU Code: 06/04/01/016'])
downloaded  dict_keys(['IDIBITIYE-ITIDION SQUARE PU Code: 06/04/01/015'])
downloaded  dict_keys(['UBE MODEL SCHOOL, TOMBI NEMBE PU Code: 06/04/01/014'])
downloaded  dict_keys(['MINGI XI SQUARE PU Code: 06/04/01/013'])
downloaded  dict_keys(['KING KOKO PARK ROUND ABOUT SQUARE PU Code: 06/04/01/012'])
downloaded  dict_keys(['AKPARANTA II OPEN SPACE PU Code: 06/04/01/011'])
downloaded  dict_keys(['ISIKARA/EGWE EWOAMA/VILLAGE SQUARE PU Code: 06/04/01/010'])
downloaded  dict_keys(['AKPARANTA/COMMUNITY SQUARE PU Code: 06/04/01/009'])
downloaded  dict_keys(['TUBOPIRI/TUBOPIRI SQUARE PU Code: 06/04/01/008'])
downloaded  dict_keys(['CH.AMAIN COMP/ST. LUKES NEMBE PU Code: 06/04/01/007'])
downloaded  dict_keys(['CH.AMAIN COMP/UAC PU Code: 06/04/01/006'])
downloaded  dict_keys